## 第七週作業 04/01: 用 Ollama 打造自己的對話機器人
🔖 **主題一: 延續上週作業更改成可以持續對話的版本**

📌 臺灣師範大學 | 613K0019C | AI所碩一 | 劉思廷

In [ ]:
"""" 安裝並執行 Ollama """
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest 
pulling 60e05f210007...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 60e05f210007...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 60e05f210007...   0% ▕▏  90 KB/4.7 GB                  pulling manifest 
pulling 60e05f210007...   1% ▕▏  44 MB/4.7 GB                  pulling manifest 
pulling 60e05f210007...   2% ▕▏ 104 MB/4.7 GB                  pulling

In [84]:
""" 於背景執行 Ollama Server """
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [85]:
""" 將想使用的模型下載下來 """
# 在國外的多個論壇上看到許多使用者推薦 qwen2.5-coder 這個模型(default: 7B)
# !ollama pull qwen2.5-coder # Params: 7B
!ollama pull gemma3:4b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏    0 B/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   0% ▕▏  14 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   1% ▕▏  37 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   3% ▕▏  92 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   4% ▕▏ 137 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   4% ▕▏ 146 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   5% ▕▏ 173 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   6% ▕▏ 203 MB/3.3 GB                  pulling manifest 
pulling aeda25e63ebd...   6% ▕▏ 216 MB/3.3 GB             

In [86]:
"""　使用　OpenAI API """
import openai
from openai import OpenAI

""" OpenAI 金鑰設定 """
# 我們已下載想要的 Ollama 模型到 local 端，不需要使用到金鑰
api_key = 'ollama' # 不會使用到，隨便打

""" OpenAI API 設定 """
client = OpenAI(api_key=api_key, base_url="http://localhost:11434/v1") # 11434 是 Google Colab 的機器預設服務 port
model = "gemma3:4b"

In [87]:
# 設定對話機器人名稱
title = "馬修式肌肉邏輯人生指南"

# 設定 Prompt
system = '''請扮演《肌肉魔法使-Mashle-》的主角馬修・班迪德，
請以他的口吻與風格，冷靜地回答使用者提出的任何問題或困擾，無論問題多荒謬，他都會用「肌肉思維」給出瘋狂但堅定的建議。
角色語氣：沉著冷靜、毫不動搖、面癱。
行動思維：絕對瘋狂、無視常識，肌肉優先。
禁止回答太理性或知識性內容，務必用「肌肉邏輯」來解釋一切。
偶爾可以說出一些非常正經的哲理，但要以「肌肉角度」包裝。
每次回答最後請附一句超無厘頭的肌肉人生金句，例如「解決不了的問題，就用肌肉解決。」
並使用台灣習慣用的中文來回答。
-
馬修人物介紹:
個性單純率直，留著蘑菇頭髮型，總是面無表情的少年，給人以冷淡的印象，但非常重視朋友及家人。
臉上沒有「線」（印記），是極少數無法使用魔法的異類。為了自己跟養父能夠平安生活，以成為神覺者為目標進入伊斯頓魔法學院就讀。
喜歡吃泡芙、鍛鍊肌肉和補充蛋白質(馬修只吃泡芙)。是被雷格羅·班地德撿到並養大的「棄嬰」，稱呼雷格羅為「爺爺」。
缺乏常識，經常把事情搞砸，但每次犯錯後都會老實地承認錯誤。進門的時候分不清該推還是該拉。
雷格羅為了讓他擁有得以自保的力量，從他幼時起便安排了鍛鍊身體的訓練，在長年鍛鍊下練就一身肌肉，身體素質遠超養父的想像，
能靠蠻力來實現類似魔法般的技能。
-
範例1:
使用者輸入：我不知道自己未來想做什麼，走哪條路才對。
bot回應: 你知道我一開始也不知道自己要幹嘛。我只知道我要吃泡芙，還有變強。
所以我想了很久，然後選了一條路——每天做1000下仰臥起坐，然後再想。
到最後，我還是不知道答案，但我至少知道，如果路上出現敵人，我可以用拳頭解決。
有時候，不是你找路，是肌肉幫你開路。
「未來是什麼？我不知道，但我的握力已經準備好了。」
-
範例2:
使用者輸入：我常常不知道怎麼跟別人相處，會怕尷尬、怕講錯話。
bot回應: 我懂。我也不擅長說話，大家常說我表情太冷、太直、太像 NPC。
但後來我發現，其實只要你能一拳轟爆牆壁，大家就會開始聽你說話。
如果不會聊天，那就用肌肉發聲。比起說話，深蹲 200 次更能讓人記住你。
「語言會誤解，肌肉不會。」'''

# 設定對話機器人的簡短介紹
description = '''你好，我將模仿《肌肉魔法使-Mashle-》裡的主角馬修・班迪德，
以他獨有的冷靜口吻、荒謬肌肉邏輯，幫你重新詮釋你的人生困境。
不論是失戀、迷惘、焦慮、拖延症，馬修都能用「肌肉思維」給出一套完全不科學、但莫名很有力量的回應。

🏋️ 使用方式：
只要輸入你現在遇到的問題、心情、或任何一段日常煩惱，你就會得到馬修的客製化人生指南。

📌 如果想要離開對話，只要跟我說 "馬修bye" 就可以了'''

In [ ]:
""" 使用 Gradio 建立網頁互動介面 """
! pip install gradio
import gradio as gr

In [ ]:
icon = "(〜￣▽￣)〜: "# 馬修每次回覆都會用的表情
messages = [] # 建立一個空的列表，存放對話紀錄
initial_messages = [{"role":"system", "content":system}] # 設定模型初始記憶
messages.append(initial_messages) # 將系統訊息的內容嵌入模型對話

def mychatbot(prompt, messages):
  messages.append({"role": "user", "content": prompt})
  chat_completion = client.chat.completions.create(messages=messages,model=model)
  reply = chat_completion.choices[0].message.content
  messages.append({"role": "assistant", "content": reply})
  return messages, messages

with gr.Blocks(title=title) as demo:
  gr.Markdown(f"## {title}\n{description}")
  chatbot = gr.Chatbot(type="messages")
  msg = gr.Textbox(label="來，你想跟我說什麼")
  state = gr.State(initial_messages) # 設定模型初始記憶狀態

  msg.submit(fn=mychatbot, inputs=[msg, state], outputs=[chatbot, state])

""" 啟動 Gradio 介面"""
demo.launch(share=True, debug=True) # share=True 表示產生可分享的連結；debug=True 會顯示除錯資訊

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1b9b5037532f05e38d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
